# GW Approximation Tutorial Notebook

In this notebook, we present an example calculation of quasiparticle energies using QuatumMASALA's `gw` module.

In [310]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [311]:
# Imports
from __future__ import annotations
import numpy as np
import sys


sys.path.append(".")

dirname = "../../../tests/bgw/silicon/cohsex/"

### Load Input Files
Input data is handled by the ``EpsInp`` class.\
The data can be provided either by constructing the ``EpsInp`` object or by reading BGW-compatible input file ``epsilon.inp``.\
The attributes have been supplied with docstrings from BerkeleyGW's input specification, so they will be accessible directly in most IDEs.

In [312]:
from qtm.gw.io_bgw.epsinp import Epsinp

# Constructing input manually
epsinp = Epsinp(epsilon_cutoff=10.0,
                use_wfn_hdf5=True,
                number_bands=8,
                write_vcoul=True,
                qpts=[[0.0,0.0,0.0]],
                is_q0=[True])

# Reading from epsilon.inp file
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')
# print(epsinp)

# There is an analogous system to read SigmaInp
from qtm.gw.io_bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')
# print(sigmainp)

### Load WfnData
Calculation of dielectric matrix requires mean field eigenfunctions. \
Wavefunction data generated from mean-field codes can be read using the ``wfn2py`` utility, which assumes that the incoming data satisfies BerkeleyGW's [`wfn_h5`](http://manual.berkeleygw.org/3.0/wfn_h5_spec/) specification. The data is stored as a `NamedTuple` object.

For reasons discussed later, we also require wavefunctions on a shifted grid to calculate dielectric matrix at $q\to 0$. This shifted grid dataset will be referred to as `wfnqdata`.

Similarly, the utilities `read_rho` and `read_vxc` can be used to read density and exchange-correlation respectively.

In [313]:
# wfn2py
from qtm.gw.io_bgw import inp
from qtm.gw.io_bgw.wfn2py import wfn2py

wfndata = wfn2py(dirname+'WFN.h5')#, wfn_ecutrho_minus_ecutwfn=epsinp.epsilon_cutoff)
print(wfndata.__doc__)

wfnqdata = wfn2py(dirname+'WFNq.h5')#, wfn_ecutrho_minus_ecutwfn=epsinp.epsilon_cutoff)
# print(wfnqdata.__doc__)

# RHO data
rho = inp.read_rho(dirname+"RHO")

# Vxc data
vxc = inp.read_vxc(dirname+"vxc.dat")

[0.91459276+0.26278404j 0.09447287+0.02714429j 0.00296139+0.00085088j
 0.0060695 +0.00174391j 0.00130582+0.00037519j]
[5.81663054e-03-9.40558288e-01j 4.24138879e-04-6.85829817e-02j
 3.62901206e-06-5.87082508e-04j 2.51956677e-05-4.07420045e-03j
 5.12124701e-06-8.28321086e-04j]
[-6.80046766e-01-4.91020637e-02j -4.41114116e-02-3.18501595e-03j
 -5.95393129e-03-4.29891403e-04j -2.24667559e-03-1.62219137e-04j
 -2.92834433e-05-2.11479737e-06j]
[-0.15657846+0.09401062j -0.0090998 +0.00546357j -0.00732237+0.00439639j
 -0.00145277+0.00087225j -0.00071016+0.00042639j]
[0.56918722+0.74880547j 0.06837594+0.08995334j 0.00306547+0.00403283j
 0.00450524+0.00592695j 0.00094637+0.00124502j]
[6.33133821e-01-6.94216332e-01j 5.02131995e-02-5.50575825e-02j
 5.34787320e-04-5.86382137e-04j 3.29207019e-03-3.60968356e-03j
 7.10060866e-04-7.78564842e-04j]
[-2.47132964e-01+8.31090956e-01j -1.60314123e-02+5.39125010e-02j
 -5.54994843e-04+1.86641068e-03j -7.94497202e-04+2.67184159e-03j
 -1.03134890e-04+3.46827095e-

### Initialize Epsilon Class

``Epsilon`` class can be initialized by either directly passing the required `quantummasala.core` objects or by passing the input objects discussed earlier.

In [314]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon(
    crystal = wfndata.crystal,
    gspace = wfndata.grho,
    kpts = wfndata.kpts,
    kptsq = wfnqdata.kpts,
    l_wfn = wfndata.l_wfn,
    l_wfnq = wfnqdata.l_wfn,
    l_gsp_wfn = wfndata.l_gk,
    l_gsp_wfnq = wfnqdata.l_gk,
    qpts = QPoints.from_cryst(wfndata.kpts.recilat, epsinp.is_q0, *epsinp.qpts),
    epsinp = epsinp,
)

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 64/64 [00:00<00:00, 17729.04it/s]


In [315]:
print(epsilon.l_wfn[0].evc.data[:3,:3])

[[ 9.14592757e-01+2.62784037e-01j  9.44728675e-02+2.71442910e-02j
   2.96138507e-03+8.50880880e-04j]
 [ 6.45053818e-09-4.62314618e-09j -4.62775051e-02+3.18821118e-02j
  -3.96432930e-03+2.73115270e-03j]
 [-2.53191638e-09-8.45071366e-10j  1.45762352e-02+4.87794633e-03j
   1.24867588e-03+4.17856600e-04j]]


The three main steps involved in the calculation have been mapped to the corresponding functions:
1.  ``matrix_elements``: Calculation of Planewave Matrix elements
    $$M_{nn'}(k,q,G) = \bra{n\,{\textbf k}{+}{\textbf q}}e^{i({\textbf q}+{\textbf G})\cdot{\textbf r}}\ket{n'\,\textbf k}$$
    where the $\textbf G$-vectors included in the calculation satisfy $|\textbf q + \textbf G|^2 < E_{\text{cut}}$ which is specified in input. \
    Since this is a convolution in k-space, the time complexity can be reduced from $N^4$ to $N^3\ln N$ by using Convolution Theorem. \
    ($N$ is roughly proportional to the number of k-vectors in the calculation.)
    $$
    M_{nn'}({\bf k},{\bf q},\{{\bf G}\}) = {\rm FFT}^{-1}\left( \phi^{*}_{n,{\bf k}+{\bf q} }({\bf r}) \phi_{n',{\bf k} }({\bf r}) \right).
    $$
    where $\phi_{n',{\bf k}}({\bf r}) = {\rm FFT}\left( \psi_{n\bf k}(\bf G)\right)$. 
    % Time complexity of this step: $O(N^3 \ln N)$
2.  ``polarizability``: Calculation of RPA polarizability matrix $P$
    $$
        P_{\textbf{GG'}}{\left({\textbf q}\;\!;0\right)}=
        \,\,{}\sum_{n}^{\textrm occ}\sum_{n'}^{\textrm emp}\sum_{{\textbf k}}
        \frac{
        \bra{n'\textbf k}e^{-i({\textbf q}+{\textbf G})\cdot{\textbf r}}\ket{n{\textbf k}{+}{\textbf q}}
        \bra{n{\textbf k}{+}{\textbf q}}e^{i({\textbf q}+{\textbf G'})\cdot{\textbf r}}\ket{n'\textbf k}
        }{E_{n{\textbf k}{+}{\textbf q}}\,{-}\,E_{n'{\textbf k}}}.
    $$
3.  ``epsilon_inverse``: Calculation of (static) Epsilon Inverse matrix
    $$
        \epsilon_{\textbf{GG'}}{\left({\textbf q}\;\!\right)}=
        \delta_{\textbf{GG'}}\,{-}\,v{\left({\textbf q}{+}{\textbf G}\right)} \,
        P_{\textbf{GG'}}{\left({\textbf q}\;\!\right)}
    $$
    where $ v(\textbf{q} + \textbf{G}) = \frac{8\pi}{\left|\textbf q + \textbf G\right|^2} $ is bare Coulomb potential, written in Rydberg units. If this formula is used as-is for the case where $|\textbf q| = |\textbf G| = 0$, the resulting $v\left({\textbf{q=0}, \textbf{G=0}}\;\!\right)$ blows up as $1/q^2$. However, for 3D gapped systems, the matrix elements $\big| M_{nn'}\left({\bf k},{\textbf{q}\to\textbf{0}},{\textbf{G=0}}\right)\big| \sim q$ cancel the Coulomb divergence and $\epsilon_{\textbf{00}}\left({\textbf q\to\textbf{0}}\;\!\right) \sim q^2/q^2$ which is a finite number. In order to calculate $\epsilon_{\textbf{00}}\left({\textbf q=\textbf{0}}\;\!\right)$, we use the scheme specified in BGW(2012), where $q=0$ is replaced with a small but non-zero value. Since matrix element calculation involves the eigenvectors $\ket{n{\textbf k}{+}{\textbf q}}$, having a non-$\Gamma$-centered $\textbf q\to 0$ point requires mean-field eigenvectors over a shifted $k$-grid.

In [316]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
            indices_gspace_sorted = sort_cryst_like_BGW(
                cryst=gkspc.g_cryst, key_array=key
            )
        else:
            key = gkspc.gk_norm2
            indices_gspace_sorted = sort_cryst_like_BGW(
                cryst=gkspc.g_cryst, key_array=key
            )

        # Calculate polarizability matrix (Memory-inefficient, but faster)
        chimat = epsilon.polarizability(next(epsilon.matrix_elements(i_q=i_q)))

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)

        # indices = epsilon.l_gq[i_q].gk_indices_tosorted
        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv, indices_gspace_sorted)
        epsilon.l_epsinv[i_q] = epsinv
        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            # indices = epsilon.l_gq[i_q].gk_indices_tosorted
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        # print("epsinv.shape", epsinv.shape,"epsref.shape", epsref.shape)
        mindim = min(*epsref.shape)#, *epsinv.shape)
        epsref = epsref[:mindim, :mindim]
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))

        epstol = 1e-15
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance", std_eps)
            print("i_q", i_q)
            # print(epsinv[0])
            # print(epsref[0])

    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)
calculate_epsilon()#numq=1)

# epsinp.no_min_fftgrid = False
# epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)
# calculate_epsilon()#numq=1)


Epsilon> q-pt index: 100%|██████████| 64/64 [00:24<00:00,  2.61it/s]


### Sigma Calculation

In [317]:
from qtm.gw.sigma import Sigma


dirname = "./scripts/results/si_6_nband272_pristine_cohsex/si_6_gw/"
outdir = "./test/tempdir_20230618_173806/"

sigma = Sigma.from_data(
    wfndata=wfndata,
    wfnqdata=wfnqdata,
    sigmainp=sigmainp,
    epsinp=epsinp,
    l_epsmats=epsilon.l_epsinv,
    rho=rho,
    vxc=vxc,
    outdir=dirname+"temp/",
)

ModuleNotFoundError: No module named 'qtm.core'

In [ ]:
print(sigma.rho.gvecs.shape)

In [ ]:
sigma.print_condition=False
sigma_x_mat = sigma.sigma_x()    
print("Sigma X GPP")
sigma.pprint_sigma_mat(np.real(sigma_x_mat.T))

In [ ]:
sigma.print_condition=False
sigma_sx_cohsex_mat = sigma.sigma_sx_static()    
print("Sigma SX COHSEX")
sigma.pprint_sigma_mat(np.real(sigma_sx_cohsex_mat.T))